# fast postprocessing a list of trajectory folders
Tim Tyree<br>
9.30.2021

In [1]:
from lib.my_initialization import *
from lib import *
# from lib.measure.unwrap_and_smooth_cu import *
# from lib.routines.unwrap_and_smooth_trajectories_cu import *
from lib.rapids_func import *
import cupy as cp, cudf, numba.cuda as cuda
import rmm
# Switch to RMM allocator
cp.cuda.set_allocator(rmm.rmm_cupy_allocator)

In [2]:
darkmode=False
if darkmode:
    # For darkmode plots
    from jupyterthemes import jtplot
    jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)

# Postprocessing trajectories from the full models

In [3]:
input_fn_lst = [
    '/home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-fk-200x200/param_set_8_ds_5.0_tmax_10_diffCoef_0.0005/Log/ic200x200.0.2_traj_sr_400_mem_0.csv',
    '/home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-3-LR/param_qu_tmax_30_Ko_5.4_diffCoef_0.0005_dt_0.5/trajectories/ic002.11_traj_sr_600_mem_0.csv',
    '/home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-fk-200x200/param_set_8_ds_5.0_tmax_30_diffCoef_0.001_dt_0.025/trajectories/ic200x200.0.8_traj_sr_600_mem_0.csv',
    '/home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-3-LR/param_qu_tmax_30_Ko_5.4_diffCoef_0.0005_dt_0.025_Vthresh_-40/trajectories/ic002.22_traj_sr_600_mem_0.csv',
    '/home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-3-LR/param_qu_tmax_30_Ko_5.4_diffCoef_0.0005_dt_0.025/trajectories/ic001.33_traj_sr_600_mem_0.csv',
    '/home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-3-LR/ds_5_param_qu_tmax_30_Ko_5.4_diffCoef_0.001/trajectories/ic001.22_traj_sr_400_mem_0.csv',
    '/home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-2/ds_5_param_set_8_fastkernel_V_0.4/trajectories/ic_200x200.001.12_traj_sr_400_mem_2.csv',
    '/home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-2/ds_5_param_set_8_fastkernel_V_0.5/trajectories/ic_200x200.001.21_traj_sr_400_mem_2.csv',
    '/home/timothytyree/Documents/GitHub/care/notebooks/Data/initial-conditions-suite-2/ds_5_param_set_8_fastkernel_V_0.6/trajectories/ic_200x200.001.23_traj_sr_400_mem_2.csv'
]

In [4]:
# input_fn=search_for_file()
# input_fn_lst.append(input_fn)
# print(input_fn_lst)

In [ ]:
for input_fn in input_fn_lst:
    df=pd.read_csv(input_fn)
    DT=np.around(get_DT(df),7)
    print(f"DT={DT} ms")
    save_dir_lst=routine_postprocess_trajectory_folder(input_fn,DT,tavg1=4, npartitions=None,
                                            width=200,
                                            height=200,
                                            use_drop_shorter_than=True,
                                            drop_shorter_than=50, #ms
                                            tmin=100., #ms
                                            pid_col='particle',
                                            t_col='t',
                                            printing=False)

DT=0.4 ms
running return_moving_average_of_pbc_trajectories_and_save on 972 files...


# Postprocessing trajectories from the particle model

In [ ]:
input_fn_lst = [
    '/home/timothytyree/Documents/GitHub/bgmc/python/data/local_results/euic_False_fc_2_r_0.1_D_0_L_10_kappa_0_varkappa_0/Log/pbc_particle_log1_log.csv',
    '/home/timothytyree/Documents/GitHub/bgmc/python/data/local_results/euic_False_fc_2_r_0.1_D_0_L_10_kappa_1500_varkappa_5/Log/pbc_particle_log2_log.csv',
    '/home/timothytyree/Documents/GitHub/bgmc/python/data/local_results/euic_False_fc_2_r_0.1_D_2_L_10_kappa_0_varkappa_0/Log/pbc_particle_log5_log.csv',
    '/home/timothytyree/Documents/GitHub/bgmc/python/data/local_results/euic_False_fc_2_r_0.1_D_2_L_10_kappa_1500_varkappa_0/Log/pbc_particle_log5_log.csv',
    '/home/timothytyree/Documents/GitHub/bgmc/python/data/local_results/euic_False_fc_2_r_0.1_D_2_L_10_kappa_1500_varkappa_5/Log/pbc_particle_log5_log.csv'
]

# folder=f'/home/timothytyree/Documents/GitHub/bgmc/python/data/local_results'
# input_fn=search_for_file(folder)
# input_fn_lst.append(input_fn)
# print(input_fn_lst)

In [ ]:
pid_col='pid_explicit'
for input_fn in input_fn_lst:
    df=pd.read_csv(input_fn)
    DT=np.around(get_DT(df,pid_col=pid_col),7)
    print(f"DT={DT} ms")
    save_dir_lst=routine_postprocess_trajectory_folder(input_fn,DT,tavg1=4, npartitions=None,
                                            width=10,
                                            height=10,
                                            use_drop_shorter_than=True,
                                            drop_shorter_than=50, #ms
                                            tmin=0., #ms
                                            pid_col=pid_col,
                                            t_col='t',
                                            printing=False)